## AntiAircraftArtillery

Given a fixed position on the ground, calculates azimuth, elevation and distance to nearby (<35km) aircraft.

***Do not use to effectively shot down planes, please!***

Libraries needed:

In [1]:
import numpy as np
from requests import get
from json import loads
from pandas import Series
from pandas.io.json import json_normalize

#### Function that calculates [ECEF](https://en.wikipedia.org/wiki/ECEF) coordinates:

In [2]:
def geocentric_coordinates(lat, long, h):
    """
    Calculates ECEF coordinates (Earth Centered, Earth Fixed) in meters.
    
    Input:
        lat, long - latitude y longitude in degrees
        h - altitude above SL in feet
        
    Output:
         - array with ECEF coordinates in meters
    """
    a = 6378137  # [m] Earth equatorial axis 
    b = 6356752.3142  # [m] Earth polar axis b 
    e = 8.1819190842622e-2  # Earth eccentricity
    
    lat = np.radians(lat)  # degrees to radians
    long = np.radians(long) # degrees to radians
    h = h * 0.3048  # feets to meters
    
    N = a / (1 - (e * np.sin(lat))**2)**(.5)

    x = (N + h) * np.cos(lat) * np.cos(long)
    y = (N + h) * np.cos(lat) * np.sin(long)
    z = (((b/a)**2) * N + h) * np.sin(lat)
    
    return np.array([x, y, z])

#### Function that calculates distance, azimuth and elevation of an aircraft with respect a reference point:

In [3]:
def dist_az_elev(lat, long, h, lat_ref, long_ref, h_ref):
    """
    Returns unit vector from aircraft to the reference point, and its module in km.
    
    Input:
        lat, long - aircraft latitude and longitude in degrees
        h - aircraft altitude above sea level in feet
        lat_ref, long_ref - reference point latitude and longitude in degrees
        h_CT - reference point altitude above sea level in feet
        
    Output:
         - distance aircraft -> reference point in km, azimuth and elevation in degrees
    """
    v = geocentric_coordinates(lat, long, h) - \
        geocentric_coordinates(lat_ref, long_ref, h_ref)
    
    unit_vector_ecef = v / np.linalg.norm(v)
    distancia = np.linalg.norm(v) / 1e3  # [km]

    sla, cla = np.sin(np.radians(lat_ref)), np.cos(np.radians(lat_ref))
    slo, clo = np.sin(np.radians(long_ref)), np.cos(np.radians(long_ref))
    
    Lht = np.array([[-sla * clo, -sla * slo, cla],
                    [-slo,       clo,        0],
                    [-cla * clo, -cla * slo, -sla]])
    
    unit_vector_ned = np.dot(Lht, unit_vector_ecef)
    azimut = np.arctan2(unit_vector_ned[1], unit_vector_ned[0])
    altura = np.arctan(-unit_vector_ned[2] / np.sqrt(unit_vector_ned[0]**2 + unit_vector_ned[1]**2))
    
    return Series([distancia, np.degrees(azimut), np.degrees(altura)])

### Data extraction from [adsbexchange](https://adsbexchange.com/)

In [4]:
# header for the request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko)'}  

# we are going to refine a little bit the window, to avoid getting all aircraft around the world
url = 'http://public-api.adsbexchange.com/VirtualRadar/AircraftList.json?lat=40.3&lng=-3.78&fDstL=0&fDstU=50'

result = get(url, headers=headers)  # give me my json!!

data_raw = result.content.decode()  # json decodification
data_json = loads(data_raw)  # json loaded!

And now it's Pandas time!

In [7]:
df_raw = json_normalize(data_json['acList'])  # aircraft list normalized
df = df_raw[np.isfinite(df_raw['Lat']) | np.isfinite(df_raw['Long'])]  # get only those with lat and long data

df = df[['GAlt', 'Spd', 'Lat', 'Long', 'Vsi', 'Trak', 
         'Reg', 'Call', 'Type', 'From', 'To']].set_index('Call')  # select the info we are interested in
df

,GAlt,Spd,Lat,Long,Vsi,Trak,Reg,Type,From,To
Call,,,,,,,,,,
ANE8398,5534,215.2,40.616089,-3.551575,1152,15.9,EC-MXA,CRJX,NaN,NaN
IBE3118,19400,371.1,40.581757,-4.025574,2688,240.8,EC-KKS,A319,"LEMD Madrid Barajas, Spain","LPPT Lisbon Portela, Portugal"
NaN,35071,422.4,40.411148,-3.374695,-64,203.3,EC-KJD,A320,NaN,NaN


## Now...AIM!

We will place our gun in the center of Madrid:

In [10]:
# Madrid coordinates as point of reference
lat_Mad, long_Mad, h_Mad = 40.4169, -3.7032, 622/0.3048  # lat, long, altitude [feet]

# lambda magic: apply the dist_az_elev function to our list of aircrafts
df[['Distance', 'Azimut', 'Elevation']] = df.apply(lambda df: dist_az_elev(df['Lat'], df['Long'], df['GAlt'], 
                                                                           lat_Mad, long_Mad, h_Mad), axis=1)

# now and finally, select those within 35km radius
df[(df['Distance'] < 35)].sort_values(by=['Distance'])

,GAlt,Spd,Lat,Long,Vsi,Trak,Reg,Type,From,To,Distance,Azimut,Elevation
Call,,,,,,,,,,,,,
ANE8398,5534,215.2,40.616089,-3.551575,1152,15.9,EC-MXA,CRJX,NaN,NaN,25.607222,30.104833,2.268005
NaN,35071,422.4,40.411148,-3.374695,-64,203.3,EC-KJD,A320,NaN,NaN,29.674152,91.205806,19.707451
IBE3118,19400,371.1,40.581757,-4.025574,2688,240.8,EC-KKS,A319,"LEMD Madrid Barajas, Spain","LPPT Lisbon Portela, Portugal",33.331668,-56.077269,8.986119
